# Compressa

Compressa is a platform that offers...

In [2]:
from langchain_compressa import CompressaEmbeddings
from langchain_compressa import ChatCompressa
from langchain_compressa import CompressaRerank

## Installation

In [ ]:
# install package
!pip install git+https://github.com/insight-stream/langchain_compressa.git

## Environment Setup

Make sure to set the following environment variables:

- COMPRESSA_API_KEY

In [1]:
import os
os.environ["COMPRESSA_API_KEY"] = "your_key_here"

## An example of a RAG pipeline via Langchain Compressa

In [ ]:
!pip install langchain langchain_core langchain_community langchain_chroma bs4

In [ ]:
import os
from langchain_compressa import CompressaEmbeddings, ChatCompressa, CompressaRerank
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma

COMPRESSA_API_KEY = os.getenv('COMPRESSA_API_KEY')

compressa_embedding = CompressaEmbeddings(api_key=COMPRESSA_API_KEY)
llm = ChatCompressa(api_key=COMPRESSA_API_KEY)

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=compressa_embedding)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

compressor = CompressaRerank(api_key=COMPRESSA_API_KEY)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

system_template = f"""You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise."""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", """Context information:

        {context}
        
        Query: {input}		
    """),
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(compression_retriever, question_answer_chain)

answ = rag_chain.invoke({"input": "how can langsmith help with testing?"})
print(answ["answer"])